In [2]:
import numpy as np 
import tensorflow as tf
import cv2
import os
import tqdm
import matplotlib.pyplot as plt
from PIL import Image 
import pickle

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras import layers


In [3]:
num_classes = 2
width = 80
height = 80

input_shape = (width, height, 1)

model = Sequential()

In [4]:
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape= input_shape))

# Add a max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 64 filters and a 3x3 kernel
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the previous layer output
model.add(Flatten())

# Add a fully connected layer with 128 neurons and 'relu' activation function
model.add(Dense(64, activation='relu'))

# Add the output layer with the number of classes and 'softmax' activation function
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [4]:
# data_dir = "C:\\Python programming\\Face Detection\\set2\\testing"
data_dir = "C:\\Python programming\\Face Detection\\set2\\full Data"



In [5]:

label = []
imggr = []

for gender in os.listdir(data_dir):

    gndrpath = os.path.join(data_dir, gender)

    for img in os.listdir(gndrpath):

        imgpath = os.path.join(gndrpath, img)

        image = cv2.imread(imgpath)
        # cv2.imshow('gray', image)

        imgr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # cv2.imshow('gray', imgr)

        imgr = cv2.resize(imgr, (80, 80))

        imgr = np.array(imgr)
        imggr.append(imgr)
        
        if gender=='male':
            label.append(1)
        else:
            label.append(0)

label = np.array(label)
f = open('C:\\Python programming\\Face Detection\\set2\\dfdemo.pickle', 'wb')
pickle.dump({'image': imggr, 'labels': label}, f)
f.close()


In [5]:
imgdata = pickle.load(open('C:\\Python programming\\Face Detection\\set2\\dfdemo.pickle', 'rb'))

data = np.array(imgdata['image'])
labels = np.array(imgdata['labels'])

X = data
y = labels

print(X.shape, y.shape)
# print(X)

(58658, 80, 80) (58658,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, shuffle=True, stratify=labels)

X_train, X_test = X_train/255, X_test/255

# X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
# X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

model.fit(X_train, y_train, batch_size= 32, epochs= 10, validation_data= (X_test, y_test))



Epoch 1/10
1467/1467 [==============================] - 289s 195ms/step - loss: 0.2778 - accuracy: 0.8845 - val_loss: 0.2038 - val_accuracy: 0.9231
Epoch 2/10
1467/1467 [==============================] - 288s 196ms/step - loss: 0.1673 - accuracy: 0.9365 - val_loss: 0.1472 - val_accuracy: 0.9433
Epoch 3/10
1467/1467 [==============================] - 289s 197ms/step - loss: 0.1371 - accuracy: 0.9487 - val_loss: 0.1457 - val_accuracy: 0.9460
Epoch 4/10
1467/1467 [==============================] - 294s 200ms/step - loss: 0.1152 - accuracy: 0.9579 - val_loss: 0.1603 - val_accuracy: 0.9378
Epoch 5/10
1467/1467 [==============================] - 290s 198ms/step - loss: 0.0983 - accuracy: 0.9641 - val_loss: 0.1419 - val_accuracy: 0.9474
Epoch 6/10
1467/1467 [==============================] - 297s 202ms/step - loss: 0.0832 - accuracy: 0.9703 - val_loss: 0.1345 - val_accuracy: 0.9529
Epoch 7/10
1467/1467 [==============================] - 296s 202ms/step - loss: 0.0695 - accuracy: 0.9752 - val_

In [7]:
gender_predict = model.predict(X_test)

predicted_classes = gender_predict.argmax(axis=1)

predicted_classes.shape
predicted_classes

367/367 [==============================] - 18s 49ms/step


array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [8]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(predicted_classes, y_test)

print(accuracy)

0.9432321854756223


In [9]:
ff = open('C:\Python programming\Face Detection\model.p', 'wb')
pickle.dump({'model' : model, 'labels': labels}, ff)
ff.close()